# Building a database for crime reports

The goal of this project is to use Postgres to build a database for storing data related with crimes that occurred in Boston. To do that, we will be working with the dataset boston.csv and will create a db called crimes_db with a table called boston_crimes inside of a schema named crimes. We will be creating 4 groups with different privileges based on what we need.

## Building the database and schema

First, we'll look at building the database and schema that we'll be using. In order to create the database, we'll need to set autocommit to True.

In [1]:
#import module for connecting to postgres
import psycopg2

#Connect to dq database and create new crime_db database
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("DROP DATABASE IF EXISTS crime_db")
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

In [2]:
#Connect to crime_db and create crimes schema
conn = psycopg2.connect("dbname=crime_db user=dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

## Gathering data for our table

Now that we've created out database, let's collect some data to add to our database

In [3]:
import csv

with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Column formatting for our new table

Before we create our table, we should review the data that we will be storing to determine the proper datatypes for each column. In order to do that, we'll create a function that takes in a CSV file and a column index to compute a Python set with all distinct values contained in the given column.

In [4]:
def get_col_value_set(csv_filename, col_index):
    col_value_set = set()
    with open(csv_filename, 'r') as file:
        next(file)
        reader = csv.reader(file)
        for row in reader:
            col_value_set.add(row[col_index])
    return col_value_set

for i in range(len(col_headers)):
    values = get_col_value_set('boston.csv', i)
    print('{}: {}'.format(col_headers[i], len(values)))

incident_number: 298329
offense_code: 219
description: 239
date: 1177
day_of_the_week: 7
lat: 18177
long: 18177


In [5]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [6]:
descriptions = get_col_value_set('boston.csv', 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


In [7]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating our boston_crimes table

Based on the number of different values and the results in the first row, let's do the following for setting up datatypes:

1. incident_number = set to integer and as the primary key based on every value being different and most likely numerical
2. offense_code = also set to integer
3. description = this should be a string, so let's make it VARCHAR(100)
4. date = this should be a date
5. day_of_the_week = given there are only 7 values and the first is "Sunday", let's enumerate this datatype
6. lat = decimal number
7. long = decimal number

In [8]:
cur.execute('''
CREATE TYPE weekday AS ENUM ('Sunday', 'Monday', 'Tuesday',
           'Wednesday', 'Thursday', 'Friday', 'Saturday');
''')
cur.execute('''
CREATE TABLE crimes.boston_crimes (
    incident_number INTEGER PRIMARY KEY,
    offense_code INTEGER,
    description VARCHAR(100),
    date DATE,
    day_of_the_week weekday,
    lat DECIMAL,
    long DECIMAL
);
''')

## Loading in the data

Now that we've created our table, we'll load in our data from the boston.csv file.

In [9]:
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)

In [10]:
cur.execute("SELECT * FROM crimes.boston_crimes")
cur.fetchmany(5)

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943')),
 (4,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.33418175'),
  Decimal('-71.07866441')),
 (5,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.27536542'),
  Decimal('-71.09036101'))]

## Establishing groups and privileges

Now that we have data in our database, let's start looking at establishing proper groups with correct privileges to create security around our database.

In [11]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

### Creating our readonly group

In [12]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")

### Creating our readwrite group

In [13]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

## Adding some users

Now that we've set up our database and groups, let's assign a user to each of the groups.

In [14]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Testing our database setup

We want to finish by testing the setup of our database by using SQL queries on the pg_roles table and the information_schema.table_privileges table.

Let's look at the following columns from the pg_roles table:

- rolname
- rolsuper
- rolcreaterole
- rolcreatedb
- rolcanlogin

For the information_schema.table_privileges, we'll just look at privileges related to our SQL queries on tables by group.

In [16]:
conn.close()

conn = psycopg2.connect("dbname=crime_db user=dq")
cur = conn.cursor()
cur.execute('''
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('readonly','readwrite','data_analyst','data_scientist');
''')
for user in cur:
    print(user)
cur.execute('''
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly','readwrite');
''')
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)
('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
